In [1]:
# Import the required modules# Impor 
%pylab inline 
import cv2
from IPython.display import clear_output
from helper import *
from vidnet_final import *
import os
torch.set_default_tensor_type('torch.FloatTensor')

Populating the interactive namespace from numpy and matplotlib


In [2]:
#Grab the input device, in this case the webcam
# You can also give path to the video file
movie_path = os.listdir('VideoData/')[np.random.randint(200)]
vid = cv2.VideoCapture('VideoData/'+movie_path)
label = re.findall('_([^_]*)_', movie_path)[0]
# Put the code in try-except statements
# Catch the keyboard exception and 
# release the camera device and 
# continue with the rest of code.
try:
    while(True):
        # Capture frame-by-frame
        ret, frame = vid.read()
        if not ret:
            # Release the Video Device if ret is false
            vid.release()
            # Message to be displayed after releasing the device
            print("Released Video Resource")
            break
        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Turn off the axis
        axis('off')
        # Title of the window
        title("Input Stream")
        # Display the frame
        imshow(frame)
        show()
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device
    print("Released Video Resource")

Released Video Resource


In [4]:
print('Label: ' + re.findall('_([^_]*)_', movie_path)[0])

Label: walking


In [5]:
frames = video3d('VideoData/'+movie_path, (110,110,16), color=False, skip=True,rand=False)

In [6]:
save_name = 'NEW_DATA_convpool_rand_noskip_nocol'

method = 'conv_pool'
input_features = 1

if method == 'single':
    timesteps_vid = 1
    timesteps_net = 1
    net = VidNet(method,6,input_features*timesteps_net,timesteps_vid,timesteps_vid)
elif method=='conv_pool':
    timesteps_vid = 16
    timesteps_net = 16
    net = VidNet(method,6,input_features*timesteps_net,1,timesteps_vid)
elif method == 'late':
    timesteps_vid = 4
    timesteps_net = 1
    net = VidNet(method,6,input_features*timesteps_net,timesteps_vid,timesteps_vid)
elif method == 'slow':
    timesteps_vid = 16
    timesteps_net = 4
    net = VidNet(method,6,input_features*timesteps_net,1,timesteps_vid)
else:
    print('define valid method')
# net = torch.load(save_name + '.pt')
net.load_state_dict(torch.load(save_name + '.pt'))
# d = torch.load(save_name + '.pt')
# net.cuda()
optimize = optim.Adam(net.parameters(), lr = 0.0001, weight_decay = .005)
optimize.load_state_dict(torch.load(save_name + '_opt.pt'))
# for state in optimize.state.values():
#     for k, v in state.items():
#         if torch.is_tensor(v):
#             state[k] = v.cuda()

In [7]:
labels = [re.findall('_([^_]*)_', c)[0] for c in os.listdir('VideoData/')]
x = list(set(labels))
dic = dict(zip(x, list(range(0,len(x)))))

In [8]:
# net.eval()
torch_frames = torch.Tensor(frames)
optimize.zero_grad()
outputs = net(torch_frames)
_, predicted = torch.max(outputs.detach(), 1)
print('Prediction: ' + [key for key,item in dic.items() if item==int(predicted.data[0])][0])

Prediction: handwaving


/Users/jenniferdawkins/Downloads/vidnet_final.py:107: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)
